# 第3章 字典和集合

字典和集合的实现均依赖于散列表，想要进一步理解集合和字典，就得先理解散列表的原理。

本章的内容大纲如下:

- 常见的字典方法

- 如何处理查找不到的键

- 标准库中dict类型的变种

- set和frozenset类型

- 散列表的工作原理

- 散列表带来的潜在影响（什么样的数据类型可作为键、不可预知的顺序，等等）

## 3.1 泛映射类型

collections.abc模块中有Mapping和MutableMapping这两个抽象基类，其作用在于为dict和其他类似的类型定义形式接口。

非抽象映射类型一般不会直接继承这些抽象基类，而是直接对dict或collections.User.Dict进行扩展。

In [ ]:
# 使用抽象基类和isinstance一起判定某个数据结构是否为广义的映射类型
import collections.abc as abc
mydict = {}
isinstance(mydict, abc.Mapping)

标准库里所有的映射类型都是利用dict来实现的，因此他们有个共同的限制，即只有可散列的数据类型才能用作这些映射里的键。

*<b>什么是可散列的数据类型？</b>*

<u>如果一个对象是可散列的，那么在这个对象的生命周期中，它的散列值是不变的，而且这个对象需要实现__hash__()方法。另外可散列的对象还要有__qe__()方法，这样才能跟其他键作比较。</u>

如果两个可散列对象是相等的，那么他们的散列值一定是一样的。

In [ ]:
tt = (1, 2, (30, 40))
hash(tt)

In [ ]:
tl = (1, 2, [30, 40])
hash(tl)

In [ ]:
tf = (1, 2, frozenset([30, 40]))
hash(tf)

In [ ]:
# 字典的构造方法
a = dict(one=1, two=2, three=3)
b = dict({'one':1, "two":2, "three":3})
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('one', 1), ('two', 2), ('three', 3)])
e = {'one':1, "two":2, "three":3}

a == b == c == d == e

## 3.2 字典推导

字典推导（dictcomp）可以从任何以键值对作为元素的可迭代对象中构建出字典。

In [ ]:
# 字典推导的应用
DIAL_CODES = [ 
    (86, 'China'),
    (91, 'India'),
    (1, 'United States'),
    (62, 'Indonesia'),
    (55, 'Brazil'),
    (92, 'Pakistan'),
    (880, 'Bangladesh'), 
    (234, 'Nigeria'),
    (7, 'Russia'),
    (81, 'Japan'),
]
country_code = {country: code for code, country in DIAL_CODES}
country_code


In [ ]:
{code: country.upper() for country, code in country_code.items() if code < 56}

## 3.3 常见的映射方法

- 用 **setdefault** 处理找不到的键 

以下程序从索引中获取单词出现的频率信息，并把它们写进对应的列表里

In [ ]:
'''创建一个从单词到期出现情况的映射'''

import sys
import re

WORD_RE = re.compile(r'\w+')

index = {}
with open(sys.argv[1], encoding = 'utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WRRD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            # 这其实是一种很不好的实现，这样写只是为了证明论点
            occurrences = index.get(word, []) ➊
            occurrences.append(location) ➋
            index[word] = occurrences ➌
# 以字母顺序打印出结果
for word in sorted(index, key=str.upper): ➍
    print(word, index[word])

In [ ]:
❶ 提取 word 出现的情况，如果还没有它的记录，返回 []。
❷ 把单词新出现的位置添加到列表的后面。
❸ 把新的列表放回字典中，这又牵扯到一次查询操作。
❹ sorted 函数的 key= 参数没有调用 str.uppper，而是把这个方法
的引用传递给 sorted 函数，这样在排序的时候，单词会被规范成统一
格式。
这是将方法用作一等函数的一个示例，第 5 章会谈到这一点。

# 此处插入 **正则表达式** 内容和 **enumerate** 函数。

## 1. enumerate函数

enumerate函数用于将一个可遍历的数据对象组合为一个索引序列，同时列出数据下标和数据，一般用在for循环中。

### 语法

`enumerate(sequence, [start=0])`

### 参数

sequence为可迭代序列，start为下标起始位置。

## 2. 正则表达式

- 正则表达式是一个特殊的字符序列，它能帮助你方便的检查一个字符串是否与某种模

式匹配。
- re模块使python语言拥有全部的正则表达式功能。

- compile函数根据一个模式字符串和可选的标志参数生成一个正则表达式对象。该对

象拥有一系列方法用于正则表达式匹配和替换。

### (1) **re.match** 函数

`re.match` 尝试从字符串的起始位置匹配一个模式，如果不是起始位置匹配成功的话，match就返回none.

#### 函数语法
`re.match(pattern, string, flags=0)`
#### 函数参数说明
| 参数 | 描述 |
|:---|:---:|
| pattern | 匹配的正则表示 |
| string  | 要匹配的字符串 |
| flags   | 	标志位，用于控制正则表达式的匹配方式，如：是否区分大小写，多行匹配等等。|

匹配成功re.match返回一个匹配的对象，否则返回None。

我们可以使用 `group(num)` 或 `groups()` 匹配对象函数来获取匹配表达式。

| 匹配对象方法 | 描述 |
|:---|:---|
|group(num=0)|匹配的整个表达式的字符串，group() 可以一次输入多个组号，在这种情况下它将返回一个包含那些组所对应值的元组。|
|groups|返回一个包含所有小组字符串的元组，从 1 到 所含的小组号。|



In [8]:
#!/usr/bin/python
import re
 
line = "Cats are smarter than dogs"
 
matchObj = re.match( r'(.*) are (.*?) .*', line, re.M|re.I)
 
if matchObj:
    print ("matchObj.group() : ", matchObj.group())
    print ("matchObj.group(1) : ", matchObj.group(1))
    print ("matchObj.group(2) : ", matchObj.group(2))
    print(matchObj.groups())
else:
    print ("No match!!")

matchObj.group() :  Cats are smarter than dogs
matchObj.group(1) :  Cats
matchObj.group(2) :  smarter
('Cats', 'smarter')


In [6]:
matchObj.group()

'Cats are smarter than dogs'

### (2) ** re.search **函数
re.search扫描整个字符串并返回第一个成功匹配。

#### 函数语法
`re.search(pattern, string, flags=0)`
#### 参数说明
同上

### re.match与re.search的区别
re.match只匹配字符串的开始，如果字符串开始不符合正则表达式，则匹配失败，函数

返回None；而re.search匹配整个字符串，直到找到一个匹配。

In [11]:
#!/usr/bin/python
import re
 
line = "Cats are smarter than dogs";
 
matchObj = re.match( r'dogs', line, re.M|re.I)
if matchObj:
    print ("match --> matchObj.group() : ", matchObj.group())
else:
    print ("No match!!")
 
matchObj = re.search( r'dogs', line, re.M|re.I)
if matchObj:
    print ("search --> searchObj.group() : ", matchObj.group())
else:
    print ("No match!!")

No match!!
search --> searchObj.group() :  dogs


## 3. 检索和替换
python的re模块提供了re.sub用于替换字符串中的匹配项。
### 语法
`re.sub(pattern, repl, string, count=0, flags=0)`
### 参数
- pattern: 正则中的模式字符串
- repl: 替换的字符串，也可为一个函数
- string: 要被查找替换的原始字符串
- count: 模式匹配后替换的最大次数，默认0表示替换所有的匹配

In [13]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-
 
import re
 
phone = "2004-959-559 # 这是一个国外电话号码"
 
# 删除字符串中的 Python注释 
num = re.sub(r'#.*$', "", phone)
print ("电话号码是: ", num)
 
# 删除非数字(-)的字符串 
num = re.sub(r'\D', "", phone)
print ("电话号码是 : ", num)

电话号码是:  2004-959-559 
电话号码是 :  2004959559


### repl参数是一个函数
以下实例中将字符串中的匹配的数字乘以2：

In [14]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-
 
import re
 
# 将匹配的数字乘以 2
def double(matched):
    value = int(matched.group('value'))
    return str(value * 2)
 
s = 'A23G4HFD567'
print(re.sub('(?P<value>\d+)', double, s))

A46G8HFD1134


## （3）** re.compile **函数